In [ ]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

# Set common default style for plots
sns.set(context='notebook', style='whitegrid', font_scale=2.0, rc={'figure.figsize': (16,16)})

# Fetch data
We have 100 voltage measurements for each of the 25 true distances.
_(The unit of voltage is some fraction of Volt, the actual value is not important.)_

In [ ]:
data = pd.read_csv('data/ir-final.csv')
data.describe()

In [ ]:
trueDists = [5,10,15,20,25,30,35,40,45,50,55,60,65,70,75,80,85,90,95,100,150,200,250,300,500]

# Voltage vs. True Distance

In [ ]:
# We only need the mean voltages (to plot them) and standard deviations (for error bars)
voltages = data.describe().loc[['mean', 'std']]
voltages.loc['dist'] = trueDists
voltages

In [ ]:
plt.errorbar(voltages.loc['dist'], voltages.loc['mean'], yerr=voltages.loc['std'],
             color='black',
             ecolor='red', barsabove=True, elinewidth=1.3, capthick=1.0, capsize=5)
plt.minorticks_on()
plt.grid(b=True, which='minor', color='grey', linestyle='--', linewidth=0.5)
plt.grid(b=True, which='major', color='grey', linestyle='-', linewidth=1.0)
plt.xlabel('True distance [cm]')
plt.ylabel('Voltage')
plt.show()

# Measured Distance vs True Distance
Here we convert the voltages to distances and look at the data again.

In [ ]:
# Voltage to distance conversion function
f = lambda x: 4800/(x-20) if x > 20 else 4800

In [ ]:
# Convert and show staticstics
data.applymap(f).describe()

In [ ]:
# Create numpy array with all the measured distances
dists = data.applymap(f).get_values()
mean_dists = dists.mean(axis=0)
print(dists.shape)
print(mean_dists.round())

The errors for large distances are not normally distributed after the conversion - their distribution is skewed significantly. Hence we cannot use standard deviation for the error bars (it would go too far down and make it look like we can get false positives even though we never did).

In [ ]:
plt.scatter(np.tile(trueDists, 100), dists.flatten(), marker='x', linewidth=0.5, color='r')
plt.minorticks_on()
plt.grid(b=True, which='minor', color='grey', linestyle='--', linewidth=0.5)
plt.grid(b=True, which='major', color='grey', linestyle='-', linewidth=1.0)
plt.xlabel('True distance [cm]')
plt.ylabel('Measured distance [cm]')
plt.title('All collected datapoints')
plt.show()

In [ ]:
plt.plot(trueDists, mean_dists, 'ko-', linewidth=1)
plt.scatter(np.tile(trueDists, 100), dists.flatten(), marker='+', s=40, linewidth=1.5, color='r')
plt.plot([-5,500], [-5,500], 'b--')

plt.xlim(-5,405)
plt.ylim(-5,405)
plt.minorticks_on()
plt.grid(b=True, which='minor', color='grey', linestyle='--', linewidth=0.5)
plt.grid(b=True, which='major', color='grey', linestyle='-', linewidth=1.0)
plt.xlabel('True distance [cm]')
plt.ylabel('Measured distance [cm]')
plt.title('Zoomed in version of the all datapoints graph')
plt.legend(('Mean measured distance', 'True distance', 'Individual datapoints'), frameon=True, fancybox=True)

plt.show()

## Adding some BS error bars
Since we can't use standard deviation, I'll try to hack it with percentiles. Basically, I take the ~3rd smallest and ~3rd largest value as the boundaries of an error bar.

In [ ]:
dist_arr = data.applymap(f).get_values()
upper = np.percentile(dist_arr, 98, axis=0) - dist_arr.mean(axis=0)
lower = np.percentile(dist_arr, 2, axis=0) - dist_arr.mean(axis=0)
print(upper)
print(lower)

In [ ]:
plt.errorbar(trueDists, mean_dists, yerr=np.array([-lower, upper]),
             fmt='ko-',
             ecolor='red', barsabove=True, elinewidth=1.5, capthick=1.0, capsize=5)
plt.plot([-5,500], [-5,500], 'b--')

plt.xlim(-5,405)
plt.ylim(-5,405)
plt.minorticks_on()
plt.grid(b=True, which='minor', color='grey', linestyle='--', linewidth=0.5)
plt.grid(b=True, which='major', color='grey', linestyle='-', linewidth=1.0)
plt.xlabel('True distance [cm]')
plt.ylabel('Measured distance [cm]')
plt.title('Measured Distance vs True Distance')
plt.legend(('True distance', 'Mean measured distance'), frameon=True, fancybox=True)

plt.show()